In [2]:
import torch
import torch.nn as nn

In [32]:
"""
Напишите функцию create_model, которая должна возвращать полносвязную нейронную сеть из двух слоев. 
На вход должно быть 100 чисел, на выход 1, посередине 10. В качестве нелинейности используйте ReLU. 
Воспользуйтесь nn.Sequential и передайте слои как последовательность
"""

def create_model():
    net = nn.Sequential(
        nn.Linear(100, 10),
        nn.ReLU(),
        nn.Linear(10, 1)
    )
    return net

In [29]:
"""
Напишите функцию train. Она должна принимать на вход нейронную сеть, даталоадер, оптимизатор и функцию потерь. 
Она должна иметь следующую сигнатуру: def train(model: nn.Module, data_loader: DataLoader, optimizer: Optimizer, loss_fn):

Внутри функции сделайте следующие шаги:

1. Переведите модель в режим обучения.

2. Проитерируйтесь по даталоадеру.

3. На каждой итерации:

    - Занулите градиенты с помощью оптимизатора

    - Сделайте проход вперед (forward pass)

    - Посчитайте ошибку

    - Сделайте проход назад (backward pass)

    - Напечатайте ошибку на текущем батче с точностью до 5 символов после запятой (только число)

    - Сделайте шаг оптимизации

Функция должна вернуть среднюю ошибку за время прохода по даталоадеру.
"""
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
from torch.optim import Optimizer

def train(model: nn.Module, data_loader: DataLoader, optimizer: Optimizer, loss_fn):
    model.train()
    loss_model = 0
    for x, y in data_loader:
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output, y)
        loss.backward()
        print(f"{loss.item():.5f}")
        loss_model += loss.item()
        optimizer.step()
    loss_model /= len(data_loader)
    return loss_model

In [14]:
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset

In [15]:
class CustomDataset(Dataset):
    def __init__(self, w_true, n_features, n_objects):
        self.X = (torch.rand(n_objects, n_features) - 0.5) * 10
        self.X *= (torch.arange(n_features) * 2 + 1)
        self.Y = self.X @ w_true + torch.randn(n_objects)

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, item):
        return self.X[item], self.Y[item]

In [16]:
n_features = 2
n_objects = 300

torch.manual_seed(0);
w_true = torch.randn(n_features)

X = (torch.rand(n_objects, n_features) - 0.5) * 10
X *= (torch.arange(n_features) * 2 + 1)
Y = (X @ w_true + torch.randn(n_objects)).unsqueeze(1)

In [17]:
dataset = CustomDataset(w_true, n_features, n_objects)

In [33]:
from torch import optim

model = create_model()
loader = DataLoader(dataset, batch_size=4, shuffle=True)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [36]:
"""
Напишите функцию evaluate. Она должна принимать на вход нейронную сеть, даталоадер и функцию потерь. 
Она должна иметь следующую сигнатуру: def evaluate(model: nn.Module, data_loader: DataLoader, loss_fn):

Внутри функции сделайте следующие шаги:

1. Переведите модель в режим инференса (применения)

2. Проитерируйтесь по даталоадеру

3. На каждой итерации:

    - Сделайте проход вперед (forward pass)

    - Посчитайте ошибку

Функция должна вернуть среднюю ошибку за время прохода по даталоадеру.
"""
@torch.no_grad()
def evaluate(model: nn.Module, data_loader: DataLoader, loss_fn):
    model.eval()
    total_loss = 0
    for x, y in data_loader:
        output = model(x)
        loss = loss_fn(output, y)
        total_loss += loss.item()
    total_loss /= len(data_loader)
    return total_loss